In [3]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv() 

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')  
database = 'fininsurance'

engine = create_engine(f'mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}')

query = """
SELECT 
    wp.cuscode AS cuscode,

    ic_ins.card_no AS card_no_ins,
    ic_ins.type AS type_ins,
    ic_ins.revoke_type_code AS revoke_type_ins,

    ic_life.card_no AS card_no_lift,
    ic_life.type AS type_lift,
    ic_life.revoke_type_code AS revoke_type_lift,

    CASE 
        WHEN DATEDIFF(CURRENT_DATE(), wp.date_active) <= 30 THEN 'active'
        ELSE 'nonactive'
    END AS status

FROM wp_users wp
LEFT JOIN tbl_ins_card ic_ins
    ON ic_ins.cuscode = wp.cuscode AND ic_ins.ins_type = 'INS'
LEFT JOIN tbl_ins_card ic_life
    ON ic_life.cuscode = wp.cuscode AND ic_life.ins_type = 'LIFE'
WHERE wp.cuscode LIKE 'FNG%'
    AND wp.name NOT LIKE '%ทดสอบ%'
    AND wp.name NOT LIKE '%test%'
    AND wp.name NOT LIKE '%เทสระบบ%'
    AND wp.name NOT LIKE '%Tes ระบบ%'
    AND wp.name NOT LIKE '%ทด่ท%'
    AND wp.name NOT LIKE '%ทด สอบ%'
    AND wp.fin_new_group NOT LIKE '%test%'
    AND wp.fin_sale NOT LIKE '%IT-TESTER%';
"""

df = pd.read_sql(query, engine)
df


,cuscode,card_no_ins,type_ins,revoke_type_ins,card_no_lift,type_lift,revoke_type_lift,status
0,FNG1911-0037,None,None,None,None,None,None,nonactive
1,FNG1912-0003,None,None,None,None,None,None,nonactive
2,FNG1912-0004,6204022777,B,,6503003176,B,,nonactive
3,FNG1912-0005,None,None,None,None,None,None,nonactive
4,FNG1912-0006,None,None,None,None,None,None,nonactive
...,...,...,...,...,...,...,...,...
139422,FNG25-140021,None,None,None,None,None,None,active
139423,FNG25-140022,6801036921,A,None,None,None,None,active
139424,FNG25-140023,None,None,None,None,None,None,nonactive
139425,FNG25-140024,None,None,None,None,None,None,active
